In [27]:
import threading
from dobot_api import DobotApiDashboard, DobotApi, DobotApiMove, MyType, alarmAlarmJsonFile
from time import sleep
import numpy as np
import re

In [28]:
current_actual = None
algorithm_queue = None
enableStatus_robot = None
robotErrorState = False
globalLockValue = threading.Lock()

In [29]:
def ConnectRobot():
    try:
        ip = "192.168.1.6"
        dashboardPort = 29999
        movePort = 30003
        feedPort = 30004
        print("Establishing connection...")
        dashboard = DobotApiDashboard(ip, dashboardPort)
        move = DobotApiMove(ip, movePort)
        feed = DobotApi(ip, feedPort)
        print(">.<Connection successful>!<")
        return dashboard, move, feed
    except Exception as e:
        print(":(Connection failed:(")
        raise e

In [30]:
dashboard, move, feed = ConnectRobot()
print("Starting enable...")
dashboard.EnableRobot()
print("Enable complete :)")

Establishing connection...
>.<Connection successful>!<
Starting enable...
Send to 192.168.1.6:29999: EnableRobot()
Receive from 192.168.1.6:29999: 0,{},EnableRobot();
Enable complete :)


In [31]:
def RunPoint(move: DobotApiMove, point_list: list):
    move.MovL(point_list[0], point_list[1], point_list[2], point_list[3])

In [32]:
point_a = [328.93, -17.07, 86.67, -4.77]

RunPoint(move, point_a)


MovL(328.930000,-17.070000,86.670000,-4.770000)
Send to 192.168.1.6:30003: MovL(328.930000,-17.070000,86.670000,-4.770000)
Receive from 192.168.1.6:30003: 0,{},MovL(328.930000,-17.070000,86.670000,-4.770000);


In [33]:
def ActivateVacuumGripper(dashboard: DobotApiDashboard, activate: bool):

    index = 1  # Assuming DO_01 corresponds to index 1
    status = 1 if activate else 0
    dashboard.DO(index, status)  # Activate or deactivate DO_01 based on status
    print(f"Vacuum Gripper {'activated' if activate else 'deactivated'}")

In [34]:
ActivateVacuumGripper(dashboard, activate=True)

Send to 192.168.1.6:29999: DO(1,1)
Receive from 192.168.1.6:29999: 0,{},DO(1,1);
Vacuum Gripper activated


In [35]:
ActivateVacuumGripper(dashboard, activate=False)

Send to 192.168.1.6:29999: DO(1,0)
Receive from 192.168.1.6:29999: 0,{},DO(1,0);
Vacuum Gripper deactivated


In [ ]:
def GetFeed(feed: DobotApi):
    global current_actual
    global algorithm_queue
    global enableStatus_robot
    global robotErrorState
    hasRead = 0
    while True:
        data = bytes()
        while hasRead < 1440:
            temp = feed.socket_dobot.recv(1440 - hasRead)
            if len(temp) > 0:
                hasRead += len(temp)
                data += temp
        hasRead = 0
        feedInfo = np.frombuffer(data, dtype=MyType)
        if hex((feedInfo['test_value'][0])) == '0x123456789abcdef':
            globalLockValue.acquire()
            # Refresh Properties
            current_actual = feedInfo["tool_vector_actual"][0]
            algorithm_queue = feedInfo['isRunQueuedCmd'][0]
            enableStatus_robot = feedInfo['EnableStatus'][0]
            robotErrorState = feedInfo['ErrorStatus'][0]
            globalLockValue.release()
        sleep(0.001)

In [37]:
def ClearRobotError(dashboard: DobotApiDashboard):
    global robotErrorState
    dataController, dataServo = alarmAlarmJsonFile()    # Read controller and servo alarm codes
    while True:
      globalLockValue.acquire()
      if robotErrorState:
                numbers = re.findall(r'-?\d+', dashboard.GetErrorID())
                numbers = [int(num) for num in numbers]
                if (numbers[0] == 0):
                  if (len(numbers) > 1):
                    for i in numbers[1:]:
                      alarmState = False
                      if i == -2:
                          print("Robot Alarm: Collision detected", i)
                          alarmState = True
                      if alarmState:
                          continue                
                      for item in dataController:
                        if i == item["id"]:
                            print("Robot Alarm: Controller error ID", i, item["zh_CN"]["description"])
                            alarmState = True
                            break 
                      if alarmState:
                          continue
                      for item in dataServo:
                        if i == item["id"]:
                            print("Robot Alarm: Servo error ID", i, item["zh_CN"]["description"])
                            break  
                    choose = input("Enter 1 to clear errors and continue operation: ")     
                    if int(choose) == 1:
                        dashboard.ClearError()
                        sleep(0.01)
                        dashboard.Continue()

      else:  
         if int(enableStatus_robot[0]) == 1 and int(algorithm_queue[0]) == 0:
            dashboard.Continue()
      globalLockValue.release()
      sleep(5)

In [42]:
def WaitArrive(point_list):
    while True:
        is_arrive = True
        globalLockValue.acquire()
        if current_actual is not None:
            for index in range(4):
                if (abs(current_actual[index] - point_list[index]) > 1):
                    is_arrive = False
            if is_arrive:
                globalLockValue.release()
                return
        globalLockValue.release()
        sleep(0.001)

In [ ]:
# Thread to continuously read feedback from the robot
feed_thread = threading.Thread(target=GetFeed, args=(feed,))
feed_thread.setDaemon(True)
feed_thread.start()

# Thread to monitor and clear robot errors
feed_thread1 = threading.Thread(target=ClearRobotError, args=(dashboard,))
feed_thread1.setDaemon(True)
feed_thread1.start()

In [41]:
point_a = [328.93, -17.07, 6.67, -4.77]

RunPoint(move, point_a)
WaitArrive(point_a)

MovL(328.930000,-17.070000,6.670000,-4.770000)
Send to 192.168.1.6:30003: MovL(328.930000,-17.070000,6.670000,-4.770000)
Receive from 192.168.1.6:30003: 0,{},MovL(328.930000,-17.070000,6.670000,-4.770000);


In [55]:
def GeneratePallet(cols, rows, layers, start_point, offset):
    pallet = []
    for layer in range(layers):  # Loop over layers (Z axis)
        for row in range(rows):  # Loop over rows (Y axis)
            for col in range(cols):  # Loop over columns (X axis)
                # Calculate the coordinate for each position
                target_point = [
                    start_point[0] + col * offset[0],  # X offset (columns)
                    start_point[1] + row * offset[1],  # Y offset (rows)
                    start_point[2] + layer * offset[2],  # Z offset (layers)
                    start_point[3]                    # R (rotation) stays the same
                ]
                pallet.append(target_point)
    return pallet

In [57]:
cols = 2
rows = 2
layers = 2
start_point = [270, -113.76, -140.59, 0]  # 시작점
offset = [35, 35, 50]  # X, Y, Z 간격 설정

pallet = GeneratePallet(cols, rows, layers, start_point, offset)

# 출력된 팔레트 확인
print("Generated 3D Pallet:")
for point in pallet:
    print(point)

Generated 3D Pallet:
[270, -113.76, -140.59, 0]
[305, -113.76, -140.59, 0]
[270, -78.76, -140.59, 0]
[305, -78.76, -140.59, 0]
[270, -113.76, -90.59, 0]
[305, -113.76, -90.59, 0]
[270, -78.76, -90.59, 0]
[305, -78.76, -90.59, 0]
